## Pre-defined Functions

We typically process data in the columns using functions in `pyspark.sql.functions`. Let us understand details about these functions in detail as part of this module.

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/1wGCNuAYr6k?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

* Let us recap about Functions or APIs to process Data Frames.
 * Projection - `select` or `withColumn` or `drop` or `selectExpr`
 * Filtering - `filter` or `where`
 * Grouping data by key and perform aggregations - `groupBy`
 * Sorting data - `sort` or `orderBy` 
* We can pass column names or literals or expressions to all the Data Frame APIs.
* Expressions include arithmetic operations, transformations using functions from `pyspark.sql.functions`.
* There are approximately 300 functions under `pyspark.sql.functions`.
* We will talk about some of the important functions used for String Manipulation, Date Manipulation etc.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Processing Column Data'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* Here are some of the examples of using functions to take care of required transformations.

In [5]:
# Reading data

orders = spark.read.csv(
    '/public/retail_db/orders',
    schema='order_id INT, order_date STRING, order_customer_id INT, order_status STRING'
)

In [6]:
# Importing functions

from pyspark.sql.functions import date_format

In [7]:
orders.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [8]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [9]:
# Function as part of projections

orders.select('*', date_format('order_date', 'yyyyMM').alias('order_month')).show()

+--------+--------------------+-----------------+---------------+-----------+
|order_id|          order_date|order_customer_id|   order_status|order_month|
+--------+--------------------+-----------------+---------------+-----------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|     201307|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|     201307|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|     201307|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|     201307|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|     201307|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|     201307|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|     201307|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|     201307|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|     201307|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT

In [10]:
orders.withColumn('order_month', date_format('order_date', 'yyyyMM')).show()

+--------+--------------------+-----------------+---------------+-----------+
|order_id|          order_date|order_customer_id|   order_status|order_month|
+--------+--------------------+-----------------+---------------+-----------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|     201307|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|     201307|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|     201307|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|     201307|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|     201307|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|     201307|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|     201307|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|     201307|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|     201307|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT

In [11]:
# Function as part of where or filter

orders. \
    filter(date_format('order_date', 'yyyyMM') == 201401). \
    show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|   25876|2014-01-01 00:00:...|             3414|PENDING_PAYMENT|
|   25877|2014-01-01 00:00:...|             5549|PENDING_PAYMENT|
|   25878|2014-01-01 00:00:...|             9084|        PENDING|
|   25879|2014-01-01 00:00:...|             5118|        PENDING|
|   25880|2014-01-01 00:00:...|            10146|       CANCELED|
|   25881|2014-01-01 00:00:...|             3205|PENDING_PAYMENT|
|   25882|2014-01-01 00:00:...|             4598|       COMPLETE|
|   25883|2014-01-01 00:00:...|            11764|        PENDING|
|   25884|2014-01-01 00:00:...|             7904|PENDING_PAYMENT|
|   25885|2014-01-01 00:00:...|             7253|        PENDING|
|   25886|2014-01-01 00:00:...|             8195|     PROCESSING|
|   25887|2014-01-01 00:00:...|            10062|        PENDING|
|   25888|

In [12]:
# Function as part of groupBy

orders. \
    groupBy(date_format('order_date', 'yyyyMM').alias('order_month')). \
    count(). \
    show()

+-----------+-----+
|order_month|count|
+-----------+-----+
|     201401| 5908|
|     201405| 5467|
|     201312| 5892|
|     201310| 5335|
|     201311| 6381|
|     201307| 1533|
|     201407| 4468|
|     201403| 5778|
|     201404| 5657|
|     201402| 5635|
|     201309| 5841|
|     201406| 5308|
|     201308| 5680|
+-----------+-----+

